In [90]:
import jax 
import jax.numpy as jnp
import numpy as np 
import pandas as pd
from cluster import BasicClusteringRecommender, KMeansRecommender

In [91]:
train_df = pd.read_csv('../preprocessed_dataset/train_dataset.csv')
test_df = pd.read_csv('../preprocessed_dataset/test_dataset.csv')

In [92]:
train_df.head()

,Unnamed: 0.1,Unnamed: 0,userID,review_count,songID,rating,user_id_idx,item_id_idx
0,0,966383,5226,10,82304,5,1115,12236
1,1,753438,23119,10,61708,5,4880,9184
2,2,1156841,7835,10,122957,5,1667,18346
3,3,503221,16895,10,110494,5,3568,16467
4,4,415592,15695,10,57161,5,3321,8548


In [93]:
train_df.describe()

,Unnamed: 0.1,Unnamed: 0,userID,review_count,songID,rating,user_id_idx,item_id_idx
count,40000.00000,4.000000e+04,40000.000000,40000.0,40000.000000,40000.000000,40000.000000,40000.000000
mean,19999.50000,6.657422e+05,11804.231675,10.0,68053.038100,4.549575,2497.230375,10150.078850
std,11547.14972,3.504432e+05,6830.278556,0.0,39515.208908,0.696171,1442.006203,5875.358351
min,0.00000,0.000000e+00,0.000000,10.0,2.000000,3.000000,0.000000,0.000000
25%,9999.75000,3.801635e+05,5835.000000,10.0,33848.750000,4.000000,1251.000000,5088.750000
50%,19999.50000,7.344685e+05,11837.000000,10.0,67838.500000,5.000000,2495.000000,10079.500000
75%,29999.25000,9.247522e+05,17732.000000,10.0,102227.000000,5.000000,3746.000000,15246.000000
max,39999.00000,1.315059e+06,23680.000000,10.0,136729.000000,5.000000,4999.000000,20360.000000


In [94]:
test_df.describe()

,Unnamed: 0,userID,review_count,songID,rating,user_id_idx,item_id_idx
count,2466.000000,2466.000000,2466.0,2466.000000,2466.000000,2466.000000,2466.000000
mean,3307.095702,11858.433496,10.0,67045.714517,4.558800,2508.562855,9999.652068
std,1912.952314,6858.378833,0.0,39385.201394,0.685987,1447.535411,5855.695673
min,0.000000,90.000000,10.0,60.000000,3.000000,17.000000,10.000000
25%,1608.000000,6005.000000,10.0,33557.750000,4.000000,1284.500000,5045.000000
50%,3355.500000,11871.500000,10.0,66685.000000,5.000000,2504.000000,9901.000000
75%,4928.250000,17832.750000,10.0,99568.750000,5.000000,3764.250000,14847.250000
max,6603.000000,23668.000000,10.0,136507.000000,5.000000,4997.000000,20337.000000


In [95]:
train_df = train_df[['rating', 'user_id_idx', 'item_id_idx']]
test_df = test_df[['rating', 'user_id_idx', 'item_id_idx']]

In [ ]:
train_df.rename(columns={'user_id_idx': 'userID', 'item_id_idx': 'songID'}, inplace=True)
test_df.rename(columns={'user_id_idx': 'userID', 'item_id_idx': 'songID'}, inplace=True)

In [ ]:
def eval_model(model, test_df):
    users = test_df.userID.unique()
    recall_scores = []
    precision_scores = []
    f1_scores = []
    n_recommendation = 3

    for user in users:
        recommendations = model.predict(user, n_recommendation)
        tp = len(test_df[test_df.userID == user & np.isin(test_df.songID, recommendations)])

        recall = tp / len(test_df[test_df.userID == user])
        precision = tp / n_recommendation
        f1 = 0 if (recall == 0 or precision == 0) else 2 / (1/recall + 1/precision)

        recall_scores.append(recall)
        precision_scores.append(precision)
        f1_scores.append(f1)

    return np.mean(recall_scores), np.mean(precision_scores), np.mean(f1_scores)

**Evaluate Basic Clustering Recommender**

In [ ]:
recommender = BasicClusteringRecommender(n_neighbor=50)

In [ ]:
recommender.fit(train_df)

In [ ]:
eval_model(recommender, test_df)

ValueError: Cannot take a larger sample than population when 'replace=False'

**Evaluate K-Means Recommdender with Minibatch**

In [ ]:
recommender = KMeansRecommender(n_neighbor=50, use_minibatch=True)

In [ ]:
recommender.fit(train_df)

ValueError: setting an array element with a sequence.

In [ ]:
eval_model(recommender)

(0.35964912280701755, 0.33660130718954245, 0.18452380952380953)

**Evaluate K-Means Recommender without Minibatch**

In [ ]:
recommender = KMeansRecommender(n_neighbor=50, use_minibatch=False)

In [ ]:
recommender.fit(train_df)

In [ ]:
eval_model(recommender, test_df)

(0.3189102564102564, 0.4011437908496733, 0.17153931339977854)